In [17]:
mat = [ 39 65 69 66 57;
        64 84 24 92 22;
        49 50 61 31 45;
        48 45 55 23 50;
        59 34 30 34 18]

5×5 Array{Int64,2}:
 39  65  69  66  57
 64  84  24  92  22
 49  50  61  31  45
 48  45  55  23  50
 59  34  30  34  18

In [27]:
function bound(rows, cols)
    return sum([minimum(mat[rows,i]) for i in cols])
end

bound (generic function with 2 methods)

In [29]:
bound([2, 3, 4, 5], [2, 3, 4, 5]) + 39

138

In [77]:
bound([3, 4, 5], [2, 4, 5]) + 39  + 24

138

In [78]:
bound([4, 5], [4, 5]) + 39  + 24 + 50

154

In [74]:
bound([5], [5]) + 39  + 24 + 31 + 45

157

In [75]:
39  + 24 + 31 + 45 + 18

157

In [38]:
using JuMP, Clp, Cbc

In [80]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel=2))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [81]:
@variable(m, x[1:4] >= 0, Int)

4-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]

In [82]:
@constraint(m, x[1]+3x[2]+2x[3]+x[4] <= 10)
@constraint(m, 5x[1]+x[2]+3x[3]+2x[4] <= 15)
@constraint(m, x[1]+x[2]+x[3]+x[4] <= 16)

x[1] + x[2] + x[3] + x[4] ≤ 16.0

In [83]:
@objective(m, Max, 5x[1]+4x[2]+4x[3]+2x[4])

5 x[1] + 4 x[2] + 4 x[3] + 2 x[4]

In [84]:
JuMP.optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -logLevel 2 -solve -quit (default strategy 1)
logLevel was changed from 1 to 2
Continuous objective value is 22.5 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 12 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I Heuristic feasibility pump took 2e-06 seconds (no good)
Cbc0045I Heuristic rounding took 0 seconds (no good)
Cbc0045I Heuristic greedy cover took 1e-06 seconds (no good)
Cbc0045I Heuristic greedy equality took 1e-06 seconds (no good)
Cbc0045I Heuristic DiveCoefficient took 2.3e-05 seconds (good)
Cbc0012I Integer solution of -18 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cbc0038I Full problem 3 rows 4 columns, reduced to 2 rows 2 columns
Cbc0028I Starting sub-tree for CbcHeuristicRINS - m

In [16]:
[JuMP.value(x[i]) for i=1:4]

4-element Array{Float64,1}:
 2.0
 2.0
 1.0
 0.0

In [34]:
mat

5×5 Array{Int64,2}:
 39  65  69  66  57
 64  84  24  92  22
 49  50  61  31  45
 48  45  55  23  50
 59  34  30  34  18

In [69]:
m2 = Model(with_optimizer(Cbc.Optimizer))
@variable(m2, x[1:5,1:5], Bin)
@objective(m2, Min, sum(x[i,j]*mat[i,j] for i in 1:5, j in 1:5))
for i in 1:5
    @constraint(m2, sum(x[i,:]) == 1)
    @constraint(m2, sum(x[:,i]) == 1)
end

In [68]:
mat[1,:]

5-element Array{Int64,1}:
 39
 65
 69
 66
 57

In [70]:
JuMP.optimize!(m2)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 154 - 0.00 seconds
Cgl0004I processed model has 10 rows, 25 columns (25 integer (25 of which binary)) and 50 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 154
Cbc0038I Before mini branch and bound, 25 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 154 - took 0.00 seconds
Cbc0012I Integer solution of 154 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 154, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 154 to 154
Probing was tried

In [71]:
for i in 1:5, j in 1:5
    if JuMP.value(x[i,j]) == 1
        println("Person $(i) task $(j)")
    end
end

Person 1 task 1
Person 2 task 3
Person 3 task 2
Person 4 task 4
Person 5 task 5
